In [1]:
!pip install transformers[torch] datasets evaluate rouge_score  wandb --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 12.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bc40d22e97a1d48bd8b16e87f1b745f90a182e1ceeca0feb173b4cb58e0c0ec5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.9.0
    Uninstalling fsspec-2023.9.0:
      Successfully uninstalled fsspec-2023.9.0

In [2]:
!pip install pandas==1.5.3
!pip install datasets==2.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 25.7 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.2
    Uninstalling pandas-2.0.2:
      Successfully uninstalled pandas-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2023.621.222118 requires jupyter-server~=1.16, but you have jupyter-server 2.6.0 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which 

In [3]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=05db76a20d50fa923e97eff46b14cddb921d39c504a25d4e2c4ee38f505bdcca
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


In [4]:
!pip install gdown
!gdown --id 

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1MKEMgJ0wiRHgm9W0eVtQlhtwryhh4yXU
To: /kaggle/working/train.csv
100%|██████████████████████████████████████| 11.1M/11.1M [00:00<00:00, 85.1MB/s]


In [5]:
import random, os

import pandas as pd
import numpy as np
import json

from tqdm._tqdm_notebook import tqdm

import torch
from transformers import (AutoTokenizer,
                          DataCollatorForSeq2Seq,
                          AutoModelForSeq2SeqLM,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer,
                          pipeline,
                          EarlyStoppingCallback)

import evaluate

from datasets import load_dataset

/tmp/ipykernel_28/403521310.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
import wandb
wandb.login()

%env WANDB_PROJECT=meeting_summarization_sweep
%env WANDB_LOG_MODEL=true

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=meeting_summarization_sweep
env: WANDB_LOG_MODEL=true


In [7]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(42)

In [8]:
# TRAIN_SOURCE = os.path.join("./train.json")
# TEST_SOURCE = os.path.join("./test.json")

In [9]:
# with open(TRAIN_SOURCE) as f:
#     TRAIN_DATA = json.loads(f.read())

# with open(TEST_SOURCE) as f:
#     TEST_DATA = json.loads(f.read())

In [10]:
# TRAIN_DATA[0]

In [11]:
# train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
# uid = 1000
# for data in TRAIN_DATA:
#     for agenda in data['context'].keys():
#         context = ''
#         for line in data['context'][agenda]:
#             context += data['context'][agenda][line]
#             context += ' '
#         train.loc[uid, 'uid'] = uid
#         train.loc[uid, 'title'] = data['title']
#         train.loc[uid, 'region'] = data['region']
#         train.loc[uid, 'context'] = context[:-1]
#         train.loc[uid, 'summary'] = data['label'][agenda]['summary']
#         uid += 1


# test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
# uid = 2000
# for data in TEST_DATA:
#     for agenda in data['context'].keys():
#         context = ''
#         for line in data['context'][agenda]:
#             context += data['context'][agenda][line]
#             context += ' '
#         test.loc[uid, 'uid'] = uid
#         test.loc[uid, 'title'] = data['title']
#         test.loc[uid, 'region'] = data['region']
#         test.loc[uid, 'context'] = context[:-1]
#         uid += 1

In [12]:
# train['text'] = train.title + ' ' + train.region + ' ' + train.context
# test['text'] = test.title + ' ' + test.region + ' ' + test.context

In [13]:
# train = train.drop(['title','region','context'], axis=1)
# train.head()

In [14]:
# test = test.drop(['title','region','context'], axis=1)
# test.head()

In [15]:
# train.to_csv('./train.csv', index=False)
# test.to_csv('./test.csv', index=False)

In [16]:
raw_dataset = load_dataset("csv", data_files='train.csv',split='train')
split_data = raw_dataset.train_test_split(.2)
dataset = split_data['test'].train_test_split(.1)
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8a51639aa6929eb6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['uid', 'summary', 'text'],
        num_rows: 539
    })
    test: Dataset({
        features: ['uid', 'summary', 'text'],
        num_rows: 60
    })
})

In [17]:
dataset['train'][0]

{'uid': 2991,
 'summary': '제253회 음성군의회 임시회 제3차 본회의 개의 선포.',
 'text': '제253회    본회의 제3차(2014.03.25.) 음성 의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제253회 임시회 제3차 본회의를 개의하겠습니다. 먼저 의회사무과장으로부터 보고가 있겠습니다. 의회사무과장입니다. 제253회 임시회 제3차 본회의와 관련하여 보고드리겠습니다. 2014년 3월 18일자로 남궁유 의원님 외 일곱 분의 의원님으로부터 흡연피해 회복을 위한 국민건강보험공단의 담배소송 촉구 결의문이 접수되었습니다. 음성군수로부터는 2014년 3월 18일자로 음성군 제증명 등 수수료 징수 조례 일부개정조례안, 음성군 재해경감대책협의회 운영 조례안, 2014년도 제1차 수시분 공유재산 관리계획안이 접수되었습니다. 이상으로 보고를 마치겠습니다. 사무과장님 수고하셨습니다.'}

In [18]:
checkpoint = "eenzeenee/t5-small-korean-summarization"
# checkpoint = 'hyunwoongko/kobart'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [19]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [22]:
rouge = evaluate.load("rouge")

In [23]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True )
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result['gen_len'] = np.mean(prediction_lens)

  return {k: round(v, 4) for k, v in result.items()}


In [24]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
# LEARNING_RATE = 2e-4
# BATCH_SIZE = 8
# WEIGHT_DECAY = 0.01
# EPOCHS = 4

In [26]:
def model_init():
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    return model

In [27]:
# confit default and sweep config
num_epochs = 4
Batch_size = 8
Learning_rate = 2e-4
Scheduler_type = "linear"
Weight_decay = 0.0

config_defaults ={
    'epochs' : num_epochs,
    'batch_size':Batch_size,
    'learning_rate':Learning_rate,
    'weight_decay':Weight_decay,
    'optimizer' :'AdamW',
    'SchedulerType': Scheduler_type
}

sweep_config ={
    'method':'random', #'grid', 'random', 'bayes'
}

parameters_dict = {
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'max':2e-1,
        'min': 2e-7
    },
    'weight_decay':  {
        'values': [0.01, 0.03, 0.1, 0.3, 0.5]
    },
    'SchedulerType':{
        'values': ["linear", "cosine", "constant"]
    },
}

sweep_config['parameters'] = parameters_dict

In [28]:
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache() 

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  8% |  2% |


In [29]:
def train(config=None):
    with wandb.init(config= config_defaults):
        config = wandb.config # set sweep configuration
        
    training_args = Seq2SeqTrainingArguments(
        output_dir="base_model",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        learning_rate= config.learning_rate,
        per_device_train_batch_size=Batch_size,
        per_device_eval_batch_size=Batch_size,
        weight_decay=config.weight_decay,
        num_train_epochs=num_epochs,
        lr_scheduler_type = config.SchedulerType,
        predict_with_generate=True,
        fp16=True,
        gradient_checkpointing=True,
        gradient_accumulation_steps = 8,
        report_to="wandb",
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
#         model_init=model_init,
    )

    trainer.train()



In [30]:
# def model_init(trial):
#     return AutoModelForSequenceClassification.from_pretrained(
#         model_args.checkpoint,
#         from_tf=bool(".ckpt" in model_args.checkpoint),
#         config=config,
#         cache_dir=model_args.cache_dir,
#         revision=model_args.model_revision,
#         use_auth_token=True if model_args.use_auth_token else None,
#     )

In [31]:
entity = "solowosophia13"
sweep_id = wandb.sweep(sweep_config, entity= entity, project="meeting_summarization_sweep")
wandb.agent(sweep_id, train, count=30)

Create sweep with ID: n22volvp
Sweep URL: https://wandb.ai/solowosophia13/meeting_summarization_sweep/sweeps/n22volvp


wandb: Agent Starting Run: t0yv3k1w with config:
wandb: 	SchedulerType: linear
wandb: 	learning_rate: 7.570884720512644e-07
wandb: 	weight_decay: 0.5
wandb: Currently logged in as: solowosophia13. Use `wandb login --relogin` to force relogin


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,1.628191,0.402600,0.221600,0.404500,0.401700,60.066700
2,No log,1.616721,0.425700,0.221600,0.426400,0.423800,59.583300
3,No log,1.615608,0.425700,0.221600,0.426400,0.423800,58.850000


...

wandb: Agent Starting Run: rkcppkhf with config:
wandb: 	SchedulerType: constant
wandb: 	learning_rate: 4.281004093470319e-05
wandb: 	weight_decay: 0.3


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,6.293359,0.000000,0.000000,0.000000,0.000000,127.000000
2,No log,6.293359,0.000000,0.000000,0.000000,0.000000,127.000000
3,No log,6.293359,0.000000,0.000000,0.000000,0.000000,127.000000


eval/gen_len,▁▁▁▁
eval/loss,▁▁▁▁
eval/rouge1,▁▁▁▁
eval/rouge2,▁▁▁▁
eval/rougeL,▁▁▁▁
eval/rougeLsum,▁▁▁▁
eval/runtime,█▁▅▂
eval/samples_per_second,▁█▄▇
eval/steps_per_second,▁█▄▇
train/epoch,▁▄▆██
train/global_step,▁▄▆██


In [32]:
# !gdown --id 

In [33]:
# test = pd.read_csv('./test.csv')

In [34]:
# summarizer = pipeline("summarization", model=model, tokenizer=tokenizer,device= device, max_length= 97)

In [35]:
# sample =test.loc[0,'text']

In [36]:
# summarizer(sample)

In [37]:
# preds = []

# for i in tqdm(range(test.shape[0])):
  
#   pred = summarizer(test['text'].iloc[i])
#   preds.append(pred)

# test.shape[0]  == len(preds)

In [38]:
# !gdown --id 1UlBIV2086lkBEWMFaLbdf_ev3m96PaBO

In [39]:
# results = []
# for idx, li  in enumerate(preds):
#     re =preds[idx][0].get('summary_text')
#     results.append(re)

                       
# results[:2]


In [40]:
# submission = pd.read_csv('./sample_submission.csv')
# submission.head()

In [41]:
# submission.shape

In [42]:
# submission['summary'] = results

In [43]:
# submission.head()

In [44]:
# submission.loc[0, 'summary']

In [45]:
# submission.to_csv('./baseline_submit.csv', index=False)

In [46]:
# pd.read_csv('./baseline_submit.csv')